In [15]:
import csv
import nltk
# nltk.download()
def load_dateset():
    X, y = [], []
    with open('domaci_data/train.csv', 'r', errors='ignore') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        next(reader, None) # Skip header
        for row in reader:
            y.append(int(row[1]))
            X.append(row[2])
    return X, y
tweet_strings_list, results_list = load_dateset()

all_tweets = "".join(tweet_strings_list)


In [16]:

from nltk.tokenize import sent_tokenize, word_tokenize

tokenz = word_tokenize(all_tweets)

In [17]:

from string import punctuation
from nltk.corpus import stopwords
def clean_tokenz(tokenz):
    
    stopwords_punctuation_list = set(stopwords.words('english')).union(set(punctuation)).union(['--','...'])
    words_clean = [w.lower() for w in tokenz if w not in stopwords_punctuation_list]
    return words_clean
words_clean = clean_tokenz(tokenz)

In [18]:
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer

vectorize = CountVectorizer()
print(vectorize.fit_transform(words_clean[17:1000:3]))

  (0, 66)	1
  (1, 222)	1
  (2, 0)	1
  (4, 145)	1
  (5, 112)	1
  (6, 194)	1
  (7, 94)	1
  (8, 3)	1
  (10, 69)	1
  (11, 113)	1
  (12, 243)	1
  (13, 117)	1
  (14, 198)	1
  (15, 36)	1
  (16, 187)	1
  (17, 90)	1
  (18, 129)	1
  (19, 102)	1
  (20, 47)	1
  (21, 106)	1
  (22, 29)	1
  (23, 54)	1
  (24, 83)	1
  (25, 185)	1
  (27, 120)	1
  :	:
  (301, 83)	1
  (303, 237)	1
  (305, 157)	1
  (306, 146)	1
  (307, 146)	1
  (308, 146)	1
  (309, 141)	1
  (310, 171)	1
  (311, 189)	1
  (312, 56)	1
  (313, 202)	1
  (314, 25)	1
  (315, 17)	1
  (316, 43)	1
  (317, 236)	1
  (318, 121)	1
  (319, 213)	1
  (320, 190)	1
  (321, 22)	1
  (322, 92)	1
  (323, 117)	1
  (325, 82)	1
  (326, 252)	1
  (326, 234)	1
  (327, 236)	1


In [19]:
most_freq_words = FreqDist(words_clean).most_common()[:500] #500 gives good results, 1000 is much slower and give +1% compparing to 500
features = []
for word in most_freq_words:
    features.append(word[0])
    
    
print(features)

['i', "'s", "n't", 'quot', "'m", 'good', 'like', 'get', 'lol', 'u', 'know', 'love', 'thanks', 'one', 'go', 'got', 'day', 'see', "'ll", 'amp', 'you', 'well', 'http', 'time', "'re", 'oh', 'think', 'it', 'im', 'haha', 'really', 'going', 'hope', 'would', 'work', 'ca', 'sorry', 'still', 'back', 'want', 'yeah', 'much', 'today', 'great', 'na', 'miss', 'need', "'ve", '2', 'could', 'right', 'new', 'yes', 'twitter', 'though', 'night', 'come', 'the', 'hey', 'fun', 'make', 'last', 'that', 'better', 'thank', 'feel', 'wish', 'sad', '3', 'lt', 'nice', 'bad', 'way', 'but', 'home', 'happy', 'and', 'never', 'ur', 'sure', 'morning', 'say', 'awesome', 'even', 'my', 'always', 'dont', 'wait', 'people', '..', 'no', 'us', 'gon', 'take', 'we', 'soon', 'next', 'let', 'week', 'tomorrow', "'d", 'cool', 'so', 'ok', 'show', 'thing', 'please', 'look', 'guys', 'something', 'follow', 'hear', 'getting', 'tonight', 'ya', 'glad', 'tell', 'find', 'yet', 'maybe', 'best', 'little', 'x', 'wan', 'just', 'thats', 'thought', 'm

In [21]:
def find_features(tweet, word_features):
    words = word_tokenize(tweet)
    words = clean_tokenz(words)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featureset =  [(find_features(tweet, features), sentiment) for tweet, sentiment in zip(tweet_strings_list,results_list)]

In [ ]:

print(len(featureset))

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
import random

#validation phase
random.shuffle(featureset)
training_set = featureset[:50000]
validation_set = featureset[50000:75000]
test_set = featureset[75000:]

classifiers = []
classifiers.append(nltk.classify.NaiveBayesClassifier)
classifiers.append(nltk.classify.SklearnClassifier(LogisticRegression()))
classifiers.append(nltk.classify.SklearnClassifier(KNeighborsClassifier()))
trained_classifers=[]
                   
for classifier in classifiers:
     trained_classifers.append(classifier.train(training_set))
    
#   classifier.show_most_informative_features(15)

accuaracy_tuples =[]
max_accuracy = 0
best_classifier = None
for classifier in trained_classifers:
    
    accuracy = nltk.classify.accuracy(classifier, validation_set)*100
    print('Accuracy for classifier: %s is %f' % (str(classifier), accuracy))
      
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        best_classifier = classifier
        
print("Classsifer %s has the best score" % str(best_classifier))
    

    

   

Accuracy for classifier: <nltk.classify.naivebayes.NaiveBayesClassifier object at 0x00000213EB65F2B0> is 71.092000
Accuracy for classifier: <SklearnClassifier(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))> is 72.084000
Accuracy for classifier: <SklearnClassifier(KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))> is 65.216000
Classsifer <SklearnClassifier(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))> has the best score


In [23]:

print("Test set accuracy percent:", (nltk.classify.accuracy(best_classifier, test_set))*100)

Test set accuracy percent: 70.95922205770539
